### Install Python SDK

In [ ]:
# Install Vertex AI LLM SDK
! pip install --user --upgrade google-cloud-aiplatform==1.47.0 langchain==0.1.14 langchain-google-vertexai==0.1.3 typing_extensions==4.9.0

# Dependencies required by Unstructured PDF loader
! sudo apt -y -qq install tesseract-ocr libtesseract-dev
! sudo apt-get -y -qq install poppler-utils
! pip install --user --upgrade unstructured==0.12.4 pdf2image==1.17.0 pytesseract==0.3.10 pdfminer.six==20221105
! pip install --user --upgrade pillow-heif==0.15.0 opencv-python==4.9.0.80 unstructured-inference==0.7.24 pikepdf==8.13.0 pypdf==4.0.1

# For Matching Engine integration dependencies (default embeddings)
! pip install --user --upgrade tensorflow_hub==0.16.1 tensorflow_text==2.15.0
! pip install sentence-transformers
! pip install -U langchain-community faiss-gpu
! pip install --upgrade --quiet  sentence_transformers > /dev/null
! pip install langchain_community
! pip install gpt4all
! pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-cloud-storage to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 121752 files and directories

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 14.8 MB/s eta 0:00:00


In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

# Import libraries

In [ ]:
import json
import textwrap

# Utils
import time
import uuid
from typing import List

import numpy as np
import vertexai

# Vertex AI
from google.cloud import aiplatform

print(f"Vertex AI SDK version: {aiplatform.__version__}")

# LangChain
import langchain

print(f"LangChain version: {langchain.__version__}")

from langchain.chains import RetrievalQA
from langchain.document_loaders import GCSDirectoryLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Import custom Matching Engine packages
# from utils.matching_engine import MatchingEngine
# from utils.matching_engine_utils import MatchingEngineUtils
# Import custom Matching Engine packages
from langchain_google_vertexai import VertexAI , VertexAIEmbeddings , VectorSearchVectorStore
import faiss
from faiss import IndexFlatL2
import numpy as np
import spacy
from langchain_community.embeddings import HuggingFaceEmbeddings
import json
import pdfplumber
from langchain_community.vectorstores import FAISS
import os
from google.colab import files
import zipfile
from langchain.document_loaders import BigQueryLoader #Class for storing a piece of text and associated metadata.
from langchain_community.embeddings import GPT4AllEmbeddings
from transformers import pipeline

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from langchain.prompts import PromptTemplate



Vertex AI SDK version: 1.47.0
LangChain version: 0.1.14


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
query = 'prolong alcohol intake impact human health?'
# docs = db.similarity_search(query)


In [ ]:
new_db = FAISS.load_local("/content/drive/MyDrive/Embeddings/GPT4AllEmbeddings/faiss_index", GPT4AllEmbeddings(),allow_dangerous_deserialization=True)
query = 'prolong alcohol intake impact human health?'

docs = new_db.similarity_search(query)

Downloading: 100%|██████████| 45.9M/45.9M [00:00<00:00, 102MiB/s] 
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 478MiB/s]


In [ ]:
docs

[Document(page_content='content: 36 Volume 18, Issue 3 NIH MedlinePlus Magazine :STIDERC Alcohol’s health effects: What you need to know rinking alcohol is so common that people may not question how even one beer, cocktail, or glass of wine could impact their As of 2021, 29.5 million people health. Alcohol is a part of cultural traditions all ages 12 and older had an alcohol around the world…and it’s also a drug that chemically use disorder in the past year. alters the body. People of all ages need to understand these effects. SOURCE: NATIONAL SURVEY ON DRUG USE AND HEALTH The National Institute on Alcohol Abuse and Alcoholism (NIAAA) has information on how alcohol The alcohol you consume resides mostly in the body’s impacts your health. It also has resources to help water. Because women tend to have less water in their bodies than men, if a woman and a man of the same those looking to change their drinking habits', metadata={'source': 'Title: alcohol_health_overview.PDF_.final_.080123

In [ ]:
new_db.similarity_search_with_score(query)

[(Document(page_content='content: 36 Volume 18, Issue 3 NIH MedlinePlus Magazine :STIDERC Alcohol’s health effects: What you need to know rinking alcohol is so common that people may not question how even one beer, cocktail, or glass of wine could impact their As of 2021, 29.5 million people health. Alcohol is a part of cultural traditions all ages 12 and older had an alcohol around the world…and it’s also a drug that chemically use disorder in the past year. alters the body. People of all ages need to understand these effects. SOURCE: NATIONAL SURVEY ON DRUG USE AND HEALTH The National Institute on Alcohol Abuse and Alcoholism (NIAAA) has information on how alcohol The alcohol you consume resides mostly in the body’s impacts your health. It also has resources to help water. Because women tend to have less water in their bodies than men, if a woman and a man of the same those looking to change their drinking habits', metadata={'source': 'Title: alcohol_health_overview.PDF_.final_.08012

# Model LLaMA2

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# converting it into list of string to generate summary
docs1=format_docs(docs)

In [ ]:

model_name="gpt2"
chat_pipeline=pipeline("text-generation",model=model_name)

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
chat_pipeline(docs1,max_length=1000)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "content: 36 Volume 18, Issue 3 NIH MedlinePlus Magazine :STIDERC Alcohol’s health effects: What you need to know rinking alcohol is so common that people may not question how even one beer, cocktail, or glass of wine could impact their As of 2021, 29.5 million people health. Alcohol is a part of cultural traditions all ages 12 and older had an alcohol around the world…and it’s also a drug that chemically use disorder in the past year. alters the body. People of all ages need to understand these effects. SOURCE: NATIONAL SURVEY ON DRUG USE AND HEALTH The National Institute on Alcohol Abuse and Alcoholism (NIAAA) has information on how alcohol The alcohol you consume resides mostly in the body’s impacts your health. It also has resources to help water. Because women tend to have less water in their bodies than men, if a woman and a man of the same those looking to change their drinking habits\n\n. Expert evidence provided for the paper showed that the only group with

# Text generator

In [ ]:
def generate_text(prompt, max_length=1000):
  input_ids = tokenizer.encode(prompt, return_tensors="pt")
  output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, temperature=0.7)
  generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
  return generated_text

In [ ]:
generate_text(docs1)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'content: 36 Volume 18, Issue 3 NIH MedlinePlus Magazine :STIDERC Alcohol’s health effects: What you need to know rinking alcohol is so common that people may not question how even one beer, cocktail, or glass of wine could impact their As of 2021, 29.5 million people health. Alcohol is a part of cultural traditions all ages 12 and older had an alcohol around the world…and it’s also a drug that chemically use disorder in the past year. alters the body. People of all ages need to understand these effects. SOURCE: NATIONAL SURVEY ON DRUG USE AND HEALTH The National Institute on Alcohol Abuse and Alcoholism (NIAAA) has information on how alcohol The alcohol you consume resides mostly in the body’s impacts your health. It also has resources to help water. Because women tend to have less water in their bodies than men, if a woman and a man of the same those looking to change their drinking habits\n\n. Expert evidence provided for the paper showed that the only group with potential to have a

# Text summarizer

In [ ]:

def summarize_text(text, max_length=1000):
  input_ids = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
  output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, temperature=0.2, early_stopping=True)
  summarized_text = tokenizer.decode(output[0], skip_special_tokens=True)
  return summarize_text

In [ ]:
from transformers import pipeline

def summarize_text(text, max_length=1000):
    """Summarize input text using a pre-trained GPT-2 model."""
    summarization_pipeline = pipeline("summarization", model="gpt2")
    summary = summarization_pipeline(text, max_length=max_length, min_length=50, do_sample=True)[0]['summary_text']
    return summary

# Example usage:
input_text = docs1
summary = summarize_text(input_text)
print("Summary:", summary)

The model 'GPT2LMHeadModel' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].
Your max_length is set to 1000, but your input_length is only 786

Summary: content: 36 Volume 18, Issue 3 NIH MedlinePlus Magazine :STIDERC Alcohol’s health effects: What you need to know rinking alcohol is so common that people may not question how even one beer, cocktail, or glass of wine could impact their As of 2021, 29.5 million people health. Alcohol is a part of cultural traditions all ages 12 and older had an alcohol around the world…and it’s also a drug that chemically use disorder in the past year. alters the body. People of all ages need to understand these effects. SOURCE: NATIONAL SURVEY ON DRUG USE AND HEALTH The National Institute on Alcohol Abuse and Alcoholism (NIAAA) has information on how alcohol The alcohol you consume resides mostly in the body’s impacts your health. It also has resources to help water. Because women tend to have less water in their bodies than men, if a woman and a man of the same those looking to change their drinking habits

. Expert evidence provided for the paper showed that the only group with potential to 

# Creating db as Retreiver

In [ ]:
# Create a retriever object from the 'db' using the 'as_retriever' method.
# This retriever is likely used for retrieving data or documents from the database.
retriever = new_db.as_retriever()
docs = retriever.invoke('prolong alcohol intake impact human health?')
docs

[Document(page_content='content: 36 Volume 18, Issue 3 NIH MedlinePlus Magazine :STIDERC Alcohol’s health effects: What you need to know rinking alcohol is so common that people may not question how even one beer, cocktail, or glass of wine could impact their As of 2021, 29.5 million people health. Alcohol is a part of cultural traditions all ages 12 and older had an alcohol around the world…and it’s also a drug that chemically use disorder in the past year. alters the body. People of all ages need to understand these effects. SOURCE: NATIONAL SURVEY ON DRUG USE AND HEALTH The National Institute on Alcohol Abuse and Alcoholism (NIAAA) has information on how alcohol The alcohol you consume resides mostly in the body’s impacts your health. It also has resources to help water. Because women tend to have less water in their bodies than men, if a woman and a man of the same those looking to change their drinking habits', metadata={'source': 'Title: alcohol_health_overview.PDF_.final_.080123

In [ ]:
docs = retriever.get_relevant_documents("What is prolong alcohol intake impact human health?")

context_summ=' '.join([str(i.page_content) for i in docs])

In [ ]:
context_summ

'content: 36 Volume 18, Issue 3 NIH MedlinePlus Magazine :STIDERC Alcohol’s health effects: What you need to know rinking alcohol is so common that people may not question how even one beer, cocktail, or glass of wine could impact their As of 2021, 29.5 million people health. Alcohol is a part of cultural traditions all ages 12 and older had an alcohol around the world…and it’s also a drug that chemically use disorder in the past year. alters the body. People of all ages need to understand these effects. SOURCE: NATIONAL SURVEY ON DRUG USE AND HEALTH The National Institute on Alcohol Abuse and Alcoholism (NIAAA) has information on how alcohol The alcohol you consume resides mostly in the body’s impacts your health. It also has resources to help water. Because women tend to have less water in their bodies than men, if a woman and a man of the same those looking to change their drinking habits content: nutrients Article The Global Impact of Alcohol Consumption on Premature Mortality and 

In [ ]:
docs = retriever.get_relevant_documents("What is e-cigarrette?")

context_summ1=' '.join([str(i.page_content) for i in docs])

In [ ]:
context_summ1

'. The advent of e-cigarettes as a device to self-administer addictive agents such as cocaine and synthetic cathinones may bring about additional adverse health effects associated with their concurrent use. This review aims to briefly describe e-cigarettes and their different generations, and their co-use with other addictive drugs as well as the use of the device as a tool to self-administer addictive drugs, such as cocaine, etc. content: Electronic cigarettes (e-cigarettes) are devices that allow the user to inhale nicotine in a vapor, and are primarily marketed as a means of quitting smoking and a less harmful replacement for traditional cigarette smoking. However, further research is needed to determine if vaping nicotine via e-cigarettes can be effective. Conversely, nicotine has been considered a gateway drug to alcohol and other addictive drugs and e-cigarettes containing nicotine may have the same effects. Previous reports have shown that e-cigarette use may open the gate for t